# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Содержание <a id='contents'></a>
1. [Загрузка данных](#start)
2. [Предобработка данных](#prepair)
3. [Ответы на вопросы](#questions)
4. [Общий вывод](#conclusions)

## 1. Загрузка данных <a id='start'></a> 

In [14]:
import pandas as pd

from pymystem3 import Mystem
from collections import Counter
from tqdm import tqdm
from warnings import fitlerwarnings

filterwarnings('ignore')

In [3]:
data_path = r'C:\Users\Vladimir\Documents\praktikum data'
df = pd.read_csv(data_path + r'\credits_data.csv')
print(df.info())
df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Вывод

Для получения ответа на поставленные вопросы необходимо сгруппировать записи по числу детей/образованию/семейному положению/ цели/уровню дохода и сравнить доли невозвращенных кредитов в соответствующих категориях. Предобработка данных должна решить следующие проблемы:
1. В столбце _days_employed_ есть некорректные значения - отрицательный стаж либо слишком большой стаж (в строке 4 - больше 900 лет). С другой стороны, этот стоблец не нужен для ответа на поставленные вопросы, пожалуй, его можно исключить из анализа (но сообщить о неполадках).
2. В столбце _education_ по-разному написан уровень образования, их надо унифицировать.
3. В столбце _total_income_ данные можно преобразовать в целочисленные - это не повлияет на качество категоризации. Кроме того, в этом столбце есть пропуски. 
4. В столбце _purpose_ необходимо унифицировать обозначения целей.
5. Нужно проверить таблицу на наличие дубликатов и избавиться от них.

[Наверх](#contents)

## 2. Предобработка данных <a id='prepair'></a>

### Обработка пропусков

In [4]:
# проверим число пропусков в столбце 'total_income'
print('Пропуски до обработки:', df['total_income'].isnull().sum())

Пропуски до обработки: 2174


In [5]:
# заполним пропуски медианным доходом каждой категории занятых
for income_type in df['income_type'].unique():
    income_median = df.loc[df['income_type'] == income_type]['total_income'].median()
    df.loc[(df['total_income'].isnull()) & (df['income_type'] == income_type), 'total_income'] = income_median

# проверим, что пропусков не осталось
print('Пропуски после обработки:', df['total_income'].isnull().sum())

Пропуски после обработки: 0


#### Вывод

Число пропусков в столбцах 'days_employed' и 'total_income' совпадает, вероятно, они возникли из-за не полностью внесенных данных о занятости (или утраты этих данных по техническим причинам). Поскольку доходы разных категорий людей в зависимости от их рода занятий существенно отличаются, пропуски были заменены на медианный доход соответствующей категории.

### Замена типа данных

In [3]:
df['total_income'] = df['total_income'].astype('int')
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB
None


#### Вывод

Тип данных столбца 'total_income' изменен на `int`. Это не повлияет на точность категоризации, но сделает выводы более наглядными и удобными в работе.

### Обработка дубликатов

In [7]:
# автоматический поиск дубликатов
print('Дубликатов до обработки:', df.duplicated().sum())
# проверим уникальные значения в трех столбцах с текстовыми данными, чтобы оценить, есть ли в них различия
print('Образование до обработки:', df['education'].unique())
print('Семейное положение до обработки:', df['family_status'].unique())
print('Род занятий до обработки:', df['income_type'].unique())

Дубликатов до обработки: 54
Образование до обработки: ['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']
Семейное положение до обработки: ['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']
Род занятий до обработки: ['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']


Поскольку различия только в регистре, приведем все значения к нижнему регистру.

In [8]:
df['education'] = df['education'].str.lower()
df['family_status'] = df['family_status'].str.lower()
print('Дубликатов после обработки:', df.duplicated().sum())
df = df.drop_duplicates().reset_index(drop=True)

Дубликатов после обработки: 71


In [9]:
# проверка
print('Образование после обработки', df['education'].unique())
print('Семейное положение после обработки', df['family_status'].unique())
print('Род занятий после обработки', df['income_type'].unique())
print('Дубликатов после удаления:', df.duplicated().sum())

Образование после обработки ['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень']
Семейное положение после обработки ['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'не женат / не замужем']
Род занятий после обработки ['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']
Дубликатов после удаления: 0


#### Вывод

Значения в столбцах _education_ и _family_status_ приведены к нижнему регистру. Автоматический подсчет дубликатов показывает, что в таблице 54 полных дубликата и 17 дубликатов, отличающихся регистром. В исходных данных нет _id_ клиентов, поэтому возникает вопрос, не могут ли дублирующиеся записи относиться к разным людям однакового возраста, рода занятий и т.д. В особенности это касается дубликатов, отличающихся регистром, что может объясняться человеческим фактором при внесении данных. С другой стороны, в таблице очень точные данные о доходе и стаже, их случайное совпадение у разных людей кажется практически невероятным, кроме, возможно, случаев, когда они совпадают из-за заполненных медианным доходом пропусков. Однако при обработке дубликатов в исходной таблице (пропуская стадию заполнения пропусков) мы получаем те же результаты. Таким образом, можно с высокой уверенностью заключить, что дубликаты возникли из-за технических проблем, а потому их можно удалить.

### Лемматизация

In [22]:
m = Mystem()
tqdm.pandas()
df['purpose_lemmas'] = df['purpose'].progress_apply(m.lemmatize)

C:\ProgramData\Anaconda3\lib\site-packages\tqdm\std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
 10%|███████▎                                                              | 2249/21454 [20:16:21<173:06:50, 32.45s/it]


KeyboardInterrupt: 

In [16]:
lemmas_total = [item for sublist in list(df['purpose_lemmas']) for item in sublist] #список всех лемм в запросах

print(Counter(lemmas_total)) 

  5%|███▉                                                                       | 1124/21454 [04:37<4:26:49,  1.27it/s]

строительство недвижимости
заняться высшим образованием
покупка жилой недвижимости
получение высшего образования
сделка с подержанным автомобилем
покупка коммерческой недвижимости
покупка жилья
получение высшего образования
жилье
покупка жилья
дополнительное образование
операции с жильем
свадьба
свадьба
покупка жилья для семьи
операции с жильем
покупка недвижимости
строительство жилой недвижимости
недвижимость
строительство недвижимости
недвижимость
операции с коммерческой недвижимостью
покупка жилья
на покупку подержанного автомобиля
ремонт жилью
автомобиль
заняться образованием
заняться высшим образованием
сыграть свадьбу
покупка коммерческой недвижимости
ремонт жилью
сыграть свадьбу
автомобиль
операции со своей недвижимостью
операции с жильем
ремонт жилью
покупка жилья
на покупку своего автомобиля
автомобили
заняться высшим образованием
на покупку подержанного автомобиля
покупка своего жилья
строительство собственной недвижимости
ремонт жилью
получение дополнительного образования
пр

 12%|█████████▏                                                                 | 2619/21454 [04:37<2:01:09,  2.59it/s]


покупка жилья для семьи
строительство собственной недвижимости
операции с жильем
на проведение свадьбы
на покупку своего автомобиля
сыграть свадьбу
операции с недвижимостью
операции с недвижимостью
покупка жилой недвижимости
на проведение свадьбы
заняться образованием
покупка своего жилья
строительство жилой недвижимости
операции со своей недвижимостью
операции с жильем
операции с недвижимостью
заняться высшим образованием
покупка жилья для семьи
операции со своей недвижимостью
покупка жилья для сдачи
покупка жилой недвижимости
недвижимость
операции со своей недвижимостью
ремонт жилью
профильное образование
получение образования
сделка с подержанным автомобилем
заняться высшим образованием
покупка жилья
ремонт жилью
на покупку своего автомобиля
профильное образование
покупка своего жилья
операции с жильем
покупка недвижимости
на покупку автомобиля
жилье
на проведение свадьбы
образование
дополнительное образование
заняться высшим образованием
покупка недвижимости
приобретение автомобил

 17%|█████████████                                                                | 3649/21454 [04:38<56:09,  5.28it/s]

...


ремонт жилью
покупка своего жилья
операции с недвижимостью
автомобили
покупка жилья для семьи
автомобиль
на покупку автомобиля
покупка жилья для семьи
покупка жилья
покупка коммерческой недвижимости
покупка жилья для семьи
сыграть свадьбу
автомобили
операции с жильем
покупка жилья
покупка коммерческой недвижимости
операции с коммерческой недвижимостью
свадьба
строительство недвижимости
строительство недвижимости
операции с жильем
сделка с автомобилем
сделка с подержанным автомобилем
заняться высшим образованием
получение дополнительного образования
операции с жильем
свой автомобиль
автомобили
ремонт жилью
операции с коммерческой недвижимостью
строительство жилой недвижимости
покупка жилой недвижимости
получение высшего образования
ремонт жилью
операции с коммерческой недвижимостью
сыграть свадьбу
операции с жильем
недвижимость
покупка жилой недвижимости
жилье
строительство недвижимости
операции с жильем
дополнительное образование
покупка жилья для семьи
жилье
покупка коммерческой недв

 96%|███████████████████████████████████████████████████████████████████████▎  | 20670/21454 [04:41<00:00, 4993.32it/s]

сыграть свадьбу
операции с коммерческой недвижимостью
операции с жильем
строительство жилой недвижимости
автомобили
автомобили
на покупку подержанного автомобиля
покупка своего жилья
свадьба
жилье
строительство жилой недвижимости
свой автомобиль
получение образования
операции с жильем
свадьба
на покупку автомобиля
покупка жилья для сдачи
приобретение автомобиля
на проведение свадьбы
автомобиль
покупка жилой недвижимости
операции с жильем
образование
строительство жилой недвижимости
заняться образованием
операции с жильем
свадьба
дополнительное образование
жилье
высшее образование
покупка жилья
операции со своей недвижимостью
жилье
образование
операции со своей недвижимостью
свадьба
ремонт жилью
сыграть свадьбу
покупка недвижимости
покупка недвижимости
недвижимость
покупка недвижимости
покупка жилья
строительство жилой недвижимости
покупка коммерческой недвижимости
на покупку подержанного автомобиля
операции со своей недвижимостью
сыграть свадьбу
недвижимость
покупка жилья
недвижимость


100%|████████████████████████████████████████████████████████████████████████████| 21454/21454 [04:41<00:00, 76.18it/s]


операции с жильем
покупка жилья
покупка жилой недвижимости
покупка коммерческой недвижимости
строительство недвижимости
ремонт жилью
ремонт жилью
сыграть свадьбу
покупка своего жилья
заняться образованием
покупка жилья для семьи
сделка с автомобилем
свой автомобиль
сыграть свадьбу
строительство собственной недвижимости
покупка жилой недвижимости
жилье
строительство недвижимости
на покупку автомобиля
на покупку своего автомобиля
дополнительное образование
операции с недвижимостью
покупка жилья для семьи
операции с коммерческой недвижимостью
строительство жилой недвижимости
покупка коммерческой недвижимости
строительство собственной недвижимости
профильное образование
на покупку подержанного автомобиля
недвижимость
покупка жилья
строительство недвижимости
образование
сделка с подержанным автомобилем
сыграть свадьбу
покупка коммерческой недвижимости
покупка коммерческой недвижимости
операции с коммерческой недвижимостью
операции с недвижимостью
сыграть свадьбу
свадьба
приобретение автомо

Таким образом, мы получили список всех лемм в запросах. Из них можно выбрать те, которые определяют цель кредита: "недвижимость", "жилье", "автомобиль", "образование", "свадьба", "строительство", "ремонт".  
По срезу 10 верхних строк в начале работы мы видели, что леммы "коммерческий", "операция", "семья" относятся к целям, определяемым леммами "жилье" и "недвижимость". Проверим, употребляются ли они в другой связи, например, "операция" в смысле "лечение".

In [17]:
count = 0
for purpose in df['purpose_lemmas']: 
    if (('операция' in purpose) or ('коммерческий' in purpose) or ('семья' in purpose) or ('сдача' in purpose)\
        or ('ремонт' in purpose)) and ('недвижимость' not in purpose) and ('жилье' not in purpose):
        count += 1
        print(purpose)
if count == 0:
    print('Необычных целей нет')

  0%|                                                                                        | 0/21454 [00:00<?, ?it/s]


ValueError: I/O operation on closed file

#### Вывод

Самая большая доля кредитов (почти половина) приходится на недвижимость, эта же категория самая разнообразная по числу способов занесения в таблицу. В ходе лемматизации были выделены пять основных категорий кредитов, которые будут исследоваться далее.

### Категоризация данных

In [ ]:
# добавим столбцы 'purpose_id' и 'purpose_type' в зависимости от типа цели
purpose_id, purpose_type = [], []
for lemmas in df['purpose_lemmas']:
    if ('недвижимость' in lemmas) or ('жилье' in lemmas) or ('ремонт' in lemmas):
        p_id = '1'
        p_type = 'недвижимость'
    if ('автомобиль' in lemmas):
        p_id = '2'
        p_type = 'автомобиль'
    if ('образование' in lemmas):
        p_id = '3'
        p_type = 'образование'
    if ('свадьба' in lemmas):
        p_id = '4'
        p_type = 'свадьба'
    purpose_id.append(p_id)
    purpose_type.append(p_type)
df['purpose_id'] = purpose_id
df['purpose_type'] = purpose_type

In [ ]:
df['income_quantile'] = pd.qcut(df['total_income'], 4, labels=False)

In [6]:
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_id,purpose_type,income_quartile
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,1,недвижимость,4
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,2,автомобиль,2
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,1,недвижимость,3
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,3,образование,4
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,4,свадьба,3
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,1,недвижимость,4
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,1,недвижимость,4
7,0,-152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,3,образование,2
8,2,-6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,4,свадьба,1
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,1,недвижимость,3


#### Вывод

Введены категории для целей покупок и доходов. Доход разделен на четыре категории по квартилям, цели - на пять групп по леммам, определяющим цель покупки.

[Наверх](#contents)

## 3. Ответьте на вопросы <a id='questions'></a>

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [ ]:
# рассчитаем средний процент невозвратов
# доля должников в столбце 'debt' равна среднему значению, т.к. все значения в столбце - 0 и 1
print('Общий % невозврата: {:.2%}'.format(df['debt'].mean()))

In [23]:
# создадим функцию, которая принимает значение столбца для категоризации и создает таблицу для ответа на вопрос, 
# в которой указано общее число заемщиков и процент должников
def debt_count(col):
    report = df.groupby(col).agg({'debt': ['mean', 'count']})
    report.columns = ['% невозврата', 'общее количество']
    report['% невозврата'] = report['% невозврата'].apply('{:.2%}'.format)
    return report       

In [7]:
debt_count('children')

Общий % невозврата: 8.12%


,% невозврата,общее количество
children,,
-1,2.13%,47
0,7.54%,14091
1,9.23%,4808
2,9.45%,2052
3,8.18%,330
4,9.76%,41
5,0.00%,9
20,10.53%,76


#### Вывод

Зависимость между наличием детей и возвратом кредита есть - бездетные чаще возвращают кредиты. Вероятно, это связано с тем, что людям, имеющим детей, труднее выделить некоторую часть семейного бюджета для регулярных выплат, поскольку у них меньше доля "лишних" денег, которые можно либо не тратить, либо потратить на что-то не необходимое в повседневной жизни (например, на выплату кредита). Кроме того, наличие детей повышает вероятность чрезвычайных трат, что также может вредить регулярным выплатам. При этом о зависимости между количеством детей и возвратом кредита однозначно судить трудно: число невозвратов растет для одного и двух детей, но значительно снижается для трех, а затем опять повышается для четырех (впрочем, число заемщиков с четырьмя детьми недостаточно велико для однозначных выводов). Кроме того, в таблице есть два очевидно невозможных значения: -1 ребенок и 20 детей (в принципе семья может иметь 20 детей, но это единичные случаи, а у нас их 76), скорее всего, это ошибки ввода или технические сбои, возможно, имелись в виду 1 и 2 ребенка соответственно. О них стоит сообщить, но едва ли они существенно влияют на результаты анализа из-за небольшого числа таких записей.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [8]:
debt_count('family_status').sort_values('% невозврата', ascending=False) 
# сортировка по % невозврата не добавлена в функцию, т.к. иногда (например, в случае с детьми) лучше сортировать по другим параметрам

,% невозврата,общее количество
family_status,,
не женат / не замужем,9.75%,2810
гражданский брак,9.35%,4151
женат / замужем,7.55%,12339
в разводе,7.11%,1195
вдовец / вдова,6.57%,959


#### Вывод

Зависимость между семейным положением и возвратом кредита в срок есть - чаще всего не возвращают кредиты неженатые/незамужние, реже всего - вдовы и вдовцы, причем разница между отдельными категориями достаточно значительная. Возможно, это объясняется тем, что люди, заключавшие официальный брак, в целом более привычны к долгосрочным обязательствам и планируют и выполняют их более ответственно (по сути, брак - это тоже долгосрочное обязательство). Возможно, это также связано со средним возрастом - люди в браке/разводе и тем более вдовы/вдовцы старше и потому имеют более стабильное положение в жизни в целом (в частности более стабильный доход), но эту гипотезу надо перепроверять расчетом зависимости возврата кредита от возраста.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [9]:
debt_count('income_quartile') 

,% невозврата,общее количество
income_quartile,,
1,7.96%,5364
2,8.73%,4409
3,8.64%,6317
4,7.14%,5364


#### Вывод

Наиболее богатые заемщики возвращают кредиты чаще всего. Однако зависимости между ростом дохода и возвращаемостью кредита нет, напротив, наиболее бедные заемщики возвращают кредиты чаще, чем две средние категории, что, возможно, связано с небольшими суммами кредитов.

- Как разные цели кредита влияют на его возврат в срок?

In [10]:
debt_count('purpose_type').sort_values('% невозврата', ascending=False) 

,% невозврата,общее количество
purpose_type,,
автомобиль,9.36%,4306
образование,9.22%,4013
свадьба,8.00%,2324
недвижимость,7.32%,10204
ремонт,5.77%,607


#### Вывод

Меньше всего невозвратов кредитов на ремонт, больше всего - на автомобиль и образование. В случае с автокредитами это, возможно, объясняется тем, что их берут более "эмоционально" и менее продуманно, покупая относительно дорогую машину, которая понравилась, недостаточно хорошо оценив экономическую сторону ее содержания - затраты на топливо (а цены на него постоянно растут), ремонт и т.д., так что в итоге кредит и содержание машины оказываются "не по средствам". В некоторой степени схожая ситуация - кредит, берущийся "на эмоциях", - это займы на свадьбу, но они возвращаются достаточно хорошо, поскольку суммы меньше (хотя и не всегда), а также многие заемщики выплачивают некоторую часть этих кредитов за счет свадебных подарков. Плохая статистика кредитов на образование может объясняться тем, что люди берут их для "инвестиции в себя", рассчитывая выплатить их за счет повышения дохода после получения образования (при смене профессии/повышении на работе и т.п.), однако этот расчет не оправдывается по самым разным причинам (неудачный выбор образования, не могут быстро найти новую работу и т.д.), что вызывает затруднения про выплате. Кредиты на недвижимость, как представляется, наиболее "надежные" из-за того, что люди  рассматривают их как значительные долгосрочные обязательства, а потому более тщательно их планируют.

[Наверх](#contents)

## 4. Общий вывод <a id='conclusions'></a>

Наличие детей влияет на погашение кредитов в срок: наименьший процент невозврата - среди людей, не имеющих детей. Семейное положение также влияет на возврат кредитов: люди, состоящие в браке, а также разведенные и вдовы/вдовцы возвращают кредиты чаще, чем неженатые/незамужние либо состоящие в гражданском браке. Цели кредита также влияют на возврат кредитов - кредиты на автомобиль и образование возвращаются реже среднего, на недвижимость и ремонт - чаще среднего.
Уровень дохода в меньшей степени влияет на возвращаемость кредитов, причем среди должников больше всего людей со средним доходом, тогда как наиболее богатые чаще всех возвращают деньги вовремя.

[Наверх](#contents)